In [ ]:
!pip install pandas 
!pip install psycopg2-binary
!pip install sqlalchemy

In [2]:
!pip install kafka-python

  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)

[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret():

    secret_name = "DBCreds"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    # Parse the secret string to get the credentials
    secret_dict = json.loads(secret)
    username = secret_dict['username']
    password = secret_dict['password']
    host = secret_dict['host']
    port = secret_dict['port']
    dbname = secret_dict['dbname']

    return username, password, host, port, dbname


(user,pswd,host,port,db) = get_secret()

In [ ]:
import json
from kafka import KafkaConsumer
import psycopg2
from psycopg2.extras import execute_values

# PostgreSQL database connection details
db_config = {
    'dbname': db,
    'user': user,
    'password': pswd,
    'host': host,
    'port': port
}

# Kafka topic
postgres_topic = "id-postgresql"
brokers = [f"{host}:9092"]

# Create a Kafka consumer
consumer = KafkaConsumer(
    postgres_topic,
    bootstrap_servers=brokers,
    value_deserializer=lambda message: json.loads(message.decode('utf-8'))
)

# Connect to PostgreSQL
try:
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    print("Connected to PostgreSQL successfully.")
except Exception as e:
    print(f"Failed to connect to PostgreSQL: {e}")
    exit()

try:
    print("Starting to consume messages.")
    for message in consumer:
        data = message.value
        print(f"Received message: {data}")
        uid = data.pop('uid')
        data.pop('outcome')
        try:
            insert_query = """
            INSERT INTO intrusion_data (uid, features)
            VALUES (%s, %s)
            """
            cursor.execute(insert_query, (uid, json.dumps(data)))
            conn.commit()
            print("Data inserted successfully.")
        except Exception as e:
            print(f"Failed to insert data: {e}")
            conn.rollback()
        
        
except KeyboardInterrupt:
    print("Consumer interrupted.")
finally:
    # Close PostgreSQL connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()
        print("PostgreSQL connection closed.")


Connected to PostgreSQL successfully.
Starting to consume messages.
Received message: {'duration': '-0.06779165192164523', 'protocol_type': '1.0', 'service': '22.0', 'flag': '9.0', 'logged_in': '2.3969771627095704', 'num_access_files': '-0.027631787343185037', 'count': '-1.5214150950430474', 'srv_count': '-1.1566389163865207', 'serror_rate': '-0.4640892825397911', 'srv_serror_rate': '-0.4635200022375929', 'same_srv_rate': '0.5369868599199825', 'srv_diff_host_rate': '-0.20363286237387915', 'dst_host_count': '-3.4515323915519476', 'dst_host_srv_count': '-1.6943128017309994', 'dst_host_same_srv_rate': '0.5993955804828853', 'dst_host_diff_srv_rate': '-0.28286639052518314', 'dst_host_same_src_port_rate': '-1.0220762525829463', 'dst_host_srv_diff_host_rate': '-0.15862913212491386', 'dst_host_serror_rate': '-0.464417155910191', 'dst_host_srv_serror_rate': '-0.4632019230839582', 'outcome': '0', 'uid': 'dbb410bd-13d2-40fb-ae37-d550a3241dbb'}
Data inserted successfully.
Received message: {'durat